<a href="https://colab.research.google.com/github/prj-07/MLMinis/blob/main/AgeDetectFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision.datasets.utils import download_url
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
import os
from torchvision.datasets import ImageFolder
import PIL
from torchvision.transforms.functional import to_pil_image
from torchvision.transforms import ToTensor
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset
import tarfile
import seaborn as sns
!pip install jovian --upgrade --quiet
import jovian
import numpy as np
from torchvision.utils import make_grid
import pandas as pd
!pip install  opendatasets --quiet
import opendatasets as od 

     |████████████████████████████████| 68 kB 4.4 MB/s 


In [2]:
!wget -q https://datahack-prod.s3.amazonaws.com/train_zip/train_DETg9GD.zip
!wget -q https://datahack-prod.s3.amazonaws.com/test_zip/test_Bh8pGW3.zip
!wget -q https://datahack-prod.s3.amazonaws.com/sample_submission/sample_submission_sDO3m7O.csv

!unzip -qo train_DETg9GD.zip
!unzip -qo test_Bh8pGW3.zip 

!rm -r test_Bh8pGW3.zip
!rm -r train_DETg9GD.zip

In [3]:
train_df = pd.read_csv('train.csv')

In [4]:
test_df = pd.read_csv('test.csv')

In [5]:
from pathlib import Path

In [6]:
train_images_path = 'Train'
test_images_path = 'Test'

In [7]:
train_images = {int(x.stem): x for x in Path(train_images_path).iterdir() if x.suffix == '.jpg'}

In [8]:
test_images = {int(x.stem): x for x in Path(test_images_path).iterdir() if x.suffix == '.jpg'}

In [9]:
train_df.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [10]:
train_df['Class'] = train_df['Class'].map({"YOUNG":0,"MIDDLE":1,"OLD":2})

In [11]:
train_df.head()

,ID,Class
0,377.jpg,1
1,17814.jpg,0
2,21283.jpg,1
3,16496.jpg,0
4,4487.jpg,1


In [12]:
##input is ID AND Output is class...whenever we have such pandas  dataframes containing input and output ...
#we separate  input and  output recall mnist data set me train_x,train_y banana 

In [13]:
X = train_df['ID'].values ##values used to convert to a numpy array 

In [14]:
y  = train_df['Class']

In [88]:
stats = ((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
train_tfms = tt.Compose([tt.Resize((16,16))
#                         ,tt.RandomCrop(512,padding=4,padding_mode='reflect')
                        # ,tt.CenterCrop(128)
#                          ,tt.ColorJitter(brightness=0.35, contrast=0.15, saturation=0.28, hue=0.22)
#                         ,tt.RandomRotation(90)
#                          ,tt.RandomPerspective()
#                         ,tt.RandomHorizontalFlip()
                        ,tt.RandomVerticalFlip(),
                         tt.RandomEqualize(p=0.4),
                        tt.RandomAutocontrast(0.3),
                        tt.RandomHorizontalFlip(p=0.3),
                        tt.AutoAugment(),
                        tt.ToTensor(),
                        tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.Resize((16,16)),
                         tt.ToTensor(),
                         tt.Normalize(*stats)])

In [89]:
def denormalize(image,means,stds):
  means = torch.tensor(means).reshape(1,3,1,1)
  print("means :",means)
  stds = torch.tensor(stds).reshape(1,3,1,1)
  print("stds :",stds)
  print("image.shape :",image.shape)
  return image*stds+means

In [90]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(60, 30))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=20).permute(1, 2, 0))
        break

In [91]:
annotations_file = y

In [92]:
from PIL import Image

In [93]:
class AgeDetectDataset(Dataset):
    def __init__(self, df, transform=None, is_test=False):
        self.df = df
        self.transform = transform
        self.files = test_images if is_test else train_images
        
    def __len__(self):
        return len(self.df)    
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_id, img_label = row['ID'], row['Class']
        labels = torch.zeros(3)
        labels[img_label] = 1
        img = os.path.join(train_images_path,img_id)
#         img = self.files[img_id] 
#         img = Image.open(img)
        img = Image.open(img,mode='r').convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, labels



In [94]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [95]:
batch_size = 16
device  = get_default_device()

In [96]:
train_ds = AgeDetectDataset(train_df, transform=train_tfms)
val_ds = AgeDetectDataset(train_df, transform=valid_tfms)
    
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=3, pin_memory=True)
    
    
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
    

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [97]:
criterion = nn.CrossEntropyLoss()

In [116]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions yaha se forward function me pass ho jaega 
        loss = F.cross_entropy(out, labels.view(-1,1)) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels.view(-1,1))   # Calculate loss
        acc = accuracy(out, labels.view(-1,1))           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
  predicts = (outputs >= 0.5)*1
  acc = torch.sum(predicts.argmax(1) == labels)
  return acc
    
    

In [117]:
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [118]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

In [119]:
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d(1), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [120]:
model = to_device(ResNet9(3, 3), device) ## 3 ---no of input channels rgb
model

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

In [121]:
history = [evaluate(model, val_dl)]
history

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


ValueError: ignored